In [ ]:
import os
import json
os.chdir('C:\\Users\\Romina\\Desktop\\thesis')

import datetime as dt
from datetime import datetime, date
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

In [ ]:
os.getcwd()

In [ ]:
data=pd.read_csv("final_matched_data_session30.csv", low_memory=False)

In [ ]:
data.head(30)
#data=data.astype(str)
data.info()

In [ ]:
counts=data.groupby('session_no').agg({'Result_id_0':'count',
                           'title_id_0':'count', 
                            'final_sub0':'count'})
counts.head(30)

print(counts['title_id_0'].astype(bool).sum(axis=0))
print(counts['final_sub0'].astype(bool).sum(axis=0))

In [ ]:
data['Advance_search']=data['q'].str.contains("AND" or "OR")
data.head(50)

In [ ]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from nltk.corpus import stopwords

In [ ]:
def data_processor(dat):
    dat=dat.apply(lambda x: x.str.replace(r"[OR, AND]", " "))
    dat=dat.apply(lambda x: x.str.lower())
    dat=dat.apply(lambda x: x.str.replace(r"\d+", ""))
    dat=dat.apply(lambda x: x.str.replace('[^\w\s]',""))
    dat=dat.apply(lambda x: x.str.replace(r"[︰-＠]", ""))
    dat=dat.apply(lambda x: x.str.replace(r"[_text_:]", ""))
    #dat=dat.drop_duplicates()
    dat=dat.drop_duplicates(subset=['q','title_id_0'])
    #dat=dat.replace('', np.nan)
    #dat=dat.dropna(subset=dat)
   # 
    return dat

In [ ]:
data['portal_view']=data['portal_view'].astype(float)
#data_final.info()

# sessions with click
data_click= data[data['portal_view']>0]
data_click=data_click[['session_no','q','title_id_0','final_sub0','final_sub1', 
                       #'final_sub2','final_sub3','final_sub4', 
                      # 'final_sub5',
                       'title_id_2','title_id_3', 'title_id_4' ]].set_index('session_no')

data_click=data_click.drop(columns=[])
data_click.info()
data_click.head(30)

In [ ]:
data_click_cl=data_processor(data_click)
data_click_cl.info()
data_click_cl.head(50)

In [ ]:
#data_click_cl=data_click_cl.drop_duplicates()
data_click_cl=data_click_cl.dropna(subset=['q','title_id_0'])
data_click_cl.head(50)
data_click_cl.isnull().sum()


In [ ]:
import nltk
#nltk.download()
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
data_click_token= data_click_cl.apply(lambda x: x.str.split())
#data_click_cl= data_click_cl['title_id_0'].apply(word_tokenize)
# tokenized_q=tokenized_q.to_list()
# tokenized_r=tokenized_r.to_list()
# tokenized=tokenized.to_list()
# print(tokenized[0:10])
# quer_tokens_click = [word for word in tokenized if word not in stopwords.words('english')]
#data_click_token.reset_index()

In [ ]:
data_click_token.head()
data_click_token.info()

In [ ]:
from os.path import join

import matplotlib.pyplot as plt
%matplotlib inline

import gensim
import numpy as np
import scipy as sp


In [ ]:
#embeddings_dict = {}
import codecs
glove_file='C:\\Users\\Romina\\Desktop\\thesis\\glove.6B.300d.txt'


In [ ]:
import time
from gensim.scripts.glove2word2vec import glove2word2vec
#from pyemd import emd
from gensim.similarities import WmdSimilarity

In [ ]:
glove_input_file = glove_file
word2vec_output_file = 'glove.6B.300d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

In [ ]:
from gensim.models import KeyedVectors
# load the Stanford GloVe model
filename = 'glove.6B.300d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)

In [ ]:
start = time.time()
model.init_sims(replace=True)
data_click_distance=[]
for i in range(0,data_click_token.shape[0]):
    distance_0= model.wmdistance(data_click_token.iloc[i,0],data_click_token.iloc[i,1])
    #data_click_token.loc[i,'distance_1'] = model.wmdistance(data_click_cl.iloc[i,0],data_click_cl.iloc[i,2])
    #data_click_token.loc[i,'distance_2'] = model.wmdistance(data_click_cl.iloc[i,0],data_click_cl.iloc[i,3])
    #data_click_token.loc[i,'distance_3'] = model.wmdistance(data_click_cl.iloc[i,0],data_click_cl.iloc[i,4])
    #data_click_token.loc[i,'distance_4'] = model.wmdistance(data_click_cl.iloc[i,0],data_click_cl.iloc[i,5])
    data_click_distance.append(distance_0)
#print ('distance = %.4f' % distance)

print ('Cell took %.2f seconds to run.' %(time.time() - start))

In [ ]:
data_click_token['distance_0']=data_click_distance
#data_click_token.info()

data_click_cl['distance_0']=data_click_distance
data_click_cl.head()
#data_click_cl.to_csv("tessst.csv")

In [ ]:
data_click_cl['distance_0'].describe()

In [ ]:
data_click_Nodistance_cl = data_click_cl[data_click_cl['distance_0'].isin([np.nan, np.inf])]
#                                               # .isinf() or data_click_distance['distance_0'].isnull()]
#df[~df.isin([np.nan, np.inf, -np.inf])                                              
# print(data_click_Nodistance_cl)
data_click_Nodistance_cl.head(30)
#data_click_Nodistance_cl.to_csv('data_click_Nodistance_cl_v2.csv')
data_click_Nodistance_cl.info()                                        

In [ ]:
import math

#print(data_click_distance)
#data_click_token.to_csv("click_Distance.csv")
data_click_distance_cl = [v for v in data_click_distance if not (math.isinf(v) or math.isnan(v) or v==0)]
#print(data_click_distance_cl)
len(data_click_distance_cl)
data_click_distance_cl.sort()
print(data_click_distance_cl)
print(a)

In [ ]:
pip install -U seaborn

In [ ]:
from matplotlib import pyplot

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# mean=df[' rating'].mean()
# median=df[' rating'].median()
plt.figure(figsize=(8,5))
#x='data_click_distance_cl',binwidth=3
ax=sns.distplot(data_click_distance_cl, kde=True);
plt.axvline(np.median(data_click_distance_cl), color='b', linestyle='--')
plt.axvline(np.mean(data_click_distance_cl), color='r', linestyle='--')

plt.legend({'Median':np.median(data_click_distance_cl), 'Mean':np.mean(data_click_distance_cl)})
ax.set(xlabel='Distance', ylabel='Density')
# pyplot.hist(y, bins, alpha=0.5, label='y')
# pyplot.legend(loc='upper right')
#pyplot.show()

In [ ]:
#sessions without click
data_noclick= data[data['portal_view'].isnull()]

data_noclick=data_noclick[['session_no','q','title_id_0','final_sub0','final_sub1', 
                           #'final_sub2','final_sub3','final_sub4',  'final_sub5',
                           'title_id_2','title_id_3', 'title_id_4' ]].set_index('session_no')
data_noclick.info()

In [ ]:
data_noclick_cl=data_processor(data_noclick)
data_noclick_cl.info()

In [ ]:
data_noclick_cl=data_noclick_cl.dropna(subset=['q','title_id_0'])
data_noclick_cl.head(50)
data_noclick_cl.isnull().sum()

In [ ]:
data_noclick_token= data_noclick_cl.apply(lambda x: x.str.split())
data_noclick_token.info()

In [ ]:
start = time.time()
data_noclick_distance=[]
for i in range(0,data_noclick_token.shape[0]):
    distance_0= model.wmdistance(data_noclick_token.iloc[i,0],data_noclick_token.iloc[i,1])
    #data_click_token.loc[i,'distance_1'] = model.wmdistance(data_click_cl.iloc[i,0],data_click_cl.iloc[i,2])
    #data_click_token.loc[i,'distance_2'] = model.wmdistance(data_click_cl.iloc[i,0],data_click_cl.iloc[i,3])
    #data_click_token.loc[i,'distance_3'] = model.wmdistance(data_click_cl.iloc[i,0],data_click_cl.iloc[i,4])
    #data_click_token.loc[i,'distance_4'] = model.wmdistance(data_click_cl.iloc[i,0],data_click_cl.iloc[i,5])
    data_noclick_distance.append(distance_0)
#print ('distance = %.4f' % distance)

print ('Cell took %.2f seconds to run.' %(time.time() - start))

In [ ]:
print(data_noclick_distance)
data_noclick_distance.sort()
print(data_noclick_distance)
#data_noclick_token.shape[0]

In [ ]:
data_noclick_cl['distance_0']=data_noclick_distance
#data_noclick_cl.to_csv("noclick.csv")
data_noclick_cl.head(30)

In [ ]:
distances=pd.concat([data_noclick_cl[['distance_0','final_sub0']],data_click_cl[['distance_0','final_sub0']]])
distances.info()
distances.head()
distances.to_csv("distances_session30-normalised.csv")

In [ ]:
data_noclick_Nodistance_cl = data_noclick_cl[data_noclick_cl['distance_0'].isin([np.nan, np.inf] )]
#                                               # .isinf() or data_click_distance['distance_0'].isnull()]
#df[~df.isin([np.nan, np.inf, -np.inf])                                              

# data_noclick_Nodistance_cl.head(30)
#data_noclick_Nodistance_cl.to_csv('data_noclick_Nodistance_cl_v2.csv')
#data_noclick_Nodistance_cl.info()    

In [ ]:
data_noclick_distance_cl = [v for v in data_noclick_distance if not (math.isinf(v) or math.isnan(v) or v==0)]
#print(data_noclick_distance_cl)
len(data_noclick_distance_cl)
print(data_noclick_distance_cl)

In [ ]:
from matplotlib import pyplot

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# mean=df[' rating'].mean()
# median=df[' rating'].median()
plt.figure(figsize=(8,5))
ax=sns.distplot(data_noclick_distance_cl, kde=True);
plt.axvline(np.median(data_noclick_distance_cl), color='b', linestyle='--')
plt.axvline(np.mean(data_noclick_distance_cl), color='r', linestyle='--')

plt.legend({'Median':np.median(data_noclick_distance_cl), 'Mean':np.mean(data_noclick_distance_cl)})
ax.set(xlabel='Distance', ylabel='Density')
# pyplot.hist(y, bins, alpha=0.5, label='y')
# pyplot.legend(loc='upper right')
#pyplot.show()

In [ ]:
data_click_distance_cl = pd.Series(data_click_distance_cl) 
data_noclick_distance_cl = pd.Series(data_noclick_distance_cl) 
print(data_click_distance_cl.mean())
print(data_noclick_distance_cl.mean())
print(len(data_click_distance_cl))
print(len(data_noclick_distance_cl))

In [ ]:
from scipy import stats
from scipy.stats import levene
statistic, p_value = levene(data_click_distance_cl, data_noclick_distance_cl, center='mean')
print(statistic, p_value)


In [ ]:
tStat, pValue = stats.ttest_ind(data_click_distance_cl, data_noclick_distance_cl, equal_var =False ) #run independent sample T-Test
print("P-Value:{0} T-Statistic:{1}".format(pValue,tStat)) #print the P-Value and the T-Statistic


In [ ]:
from statistics import mean, stdev
from math import sqrt

# test conditions

cohens_d = (mean(data_click_distance_cl) - mean(data_noclick_distance_cl)) / (sqrt((stdev(data_click_distance_cl) ** 2 + stdev(data_noclick_distance_cl) ** 2) / 2))

print(cohens_d)

In [ ]:
#merge this now to the query topics data on topic to append the top 3 category- for the sessions with click first
data_click_Nodistance_cl.head()